# Areen Zainab
# 22i-1115
# H


# Poker Hand Optimization using Simulated Annealing

The question statement is given on the exam paper. As mentioned there, here is an image visualizing winning poker hands 

<img src="https://media.istockphoto.com/id/1303667887/vector/texas-holdem-poker-hand-rankings-combination-set-vector-green-background.jpg?s=612x612&w=0&k=20&c=Ax8V0VbLdPBuwS1Ibp7jzPtRAoLLd-kdhB3HIk0zyjw=">

# The Boltzmann prbability
$$
P = e^{-\frac{\Delta E}{T}}
$$

- Where is $e$ the exponent
- $\Delta E$ is the change in energy
- $T$ is the current temperature


In [ ]:
math.exp(delta_e / temp)

In [ ]:
import random
import math

# Define Poker Hand Rankings
HAND_RANKINGS = {
    "High Card": 1,
    "One Pair": 2,
    "Two Pair": 3,
    "Three of a Kind": 4,
    "Straight": 5,
    "Flush": 6,
    "Full House": 7,
    "Four of a Kind": 8,
    "Straight Flush": 9,
    "Royal Flush": 10
}

# Create a standard deck of 52 cards
SUITS = ["Hearts", "Diamonds", "Clubs", "Spades"]
VALUES = {str(i): i for i in range(2, 11)}
VALUES.update({"Jack": 11, "Queen": 12, "King": 13, "Ace": 14})

DECK = [(value, suit) for value in VALUES.keys() for suit in SUITS]

#fucntion to generate a random hand
def generate_hand(deck, size=5):
    """Generates a random poker hand from the deck."""
    return random.sample(deck, size)


#function to rank the hands based on the rules of poker. This will act as your energy function. This will return the handrankings
def hand_rank(hand):
    
    values = sorted([VALUES[card[0]] for card in hand]) #sorts the cards in the hand
    suits = [card[1] for card in hand]

    # Flush ko dekho pehly-->5 same suit kyt cards 
    if len(set(suits)) == 1:
        if values == [10, 11, 12, 13, 14]:
            return HAND_RANKINGS["Royal Flush"]
        elif values == list(range(min(values), max(values)+1)):
            return HAND_RANKINGS["Straight Flush"] #consecutive cards same suit but not highest :((
        else:
            return HAND_RANKINGS["Flush"] #agar suit same hai lakin not consecutive 
        
    # Straight ko check karo --> consective cards koi bhi rank
    if values == list(range(min(values), max(values)+1)) :
        return HAND_RANKINGS["Straight"]
    
    # Four of a Kind ky pas 4 crads same rank ky 
    if len(set(values)) == 2 :
        return HAND_RANKINGS["Four of a Kind"]
    
    # Full House ky pas 3 same rank ky aur 2 same rank ky
    if len(set(values)) == 2 or (values[0] == values[1] and values[1] == values[2] and values[2] == values[3]) or (values[1] == values[2] and values[2] == values[3] and values[3] == values[4]):
        return HAND_RANKINGS["Full House"]
    
    # Three of a Kind--> 2 cards same rank ky
    if len(set(values)) == 3 or (values[0] == values[1] and values[1] == values[2]) or (values[2] == values[3] and values[3] == values[4]):
        return HAND_RANKINGS["Three of a Kind"]
    
    # Two Pair mei 2 pairs same rank ky
    if len(set(values)) == 3 or (values[0] == values[1] and values[2] == values[3]) or (values[1] == values[2] and values[3] == values[4]) or (values[0] == values[1] and values[3] == values[4]):
        return HAND_RANKINGS["Two Pair"]
    
    # One Pair same rank ka 
    if len(set(values)) == 4 or (values[0] == values[1]) or (values[1] == values[2]) or (values[2] == values[3]) or (values[3] == values[4]):
        return HAND_RANKINGS["One Pair"]
    
    # High Card jab no combination 
    return HAND_RANKINGS["High Card"]




"""Optimizes a poker hand using Simulated Annealing."""
def simulated_annealing(initial_hand, extra_pool, max_iterations=1000, start_temp=100, cooling_rate=0.99):
    
    current_hand = initial_hand[:]
    current_score = hand_rank(current_hand)
    #print("\n current score: ", current_score)
    temp = start_temp
    
    best_hand = current_hand[:]
    best_score = current_score

    for _ in range(max_iterations):
        # Candidate Move generate karo and swapping allow karo 3 dafa
        new_hand = current_hand[:]
        swap_count = random.randint(1, 3)
        swap_indices = random.sample(range(5), swap_count)
        for i in swap_indices:
            new_hand[i] = random.choice(extra_pool)

        # Evaluate Candidate Move
        new_score = hand_rank(new_hand)
        delta_e = new_score - current_score
        

        # Accept or Reject Candidate Move
        if delta_e > 0 or random.random() < math.exp(delta_e / temp):
            current_hand = new_hand
            current_score = new_score

            if current_score > best_score:
                best_hand = current_hand[:]
                best_score = current_score

        # Update Temperature
        temp *= cooling_rate
        
        if temp < 1e-6:
            break

    return best_hand, best_score

# Main Execution
if __name__ == "__main__":
    deck = DECK[:]
    random.shuffle(deck)

    # Generate Initial Hand and Extra Pool
    initial_hand = generate_hand(deck, 5)
    remaining_deck = [card for card in deck if card not in initial_hand]
    extra_pool = generate_hand(remaining_deck, 10) #we take out 10 cards to allow for controlled modifications 

    print(f"Initial Hand: {initial_hand}, Rank: {hand_rank(initial_hand)}")
    print(f"Extra Pool: {extra_pool}")

    # Run Simulated Annealing Optimization
    optimized_hand, final_rank = simulated_annealing(initial_hand, extra_pool)

    print("\nFinal Optimized Hand:")
    print(f"Hand: {optimized_hand}, Rank: {final_rank}")


Initial Hand: [('3', 'Clubs'), ('Queen', 'Hearts'), ('Queen', 'Clubs'), ('King', 'Hearts'), ('9', 'Clubs')], Rank: 2
Extra Pool: [('King', 'Clubs'), ('3', 'Hearts'), ('Jack', 'Hearts'), ('9', 'Spades'), ('King', 'Spades'), ('2', 'Diamonds'), ('6', 'Spades'), ('6', 'Diamonds'), ('Ace', 'Diamonds'), ('7', 'Spades')]

 current score:  2

Final Optimized Hand:
Hand: [('9', 'Spades'), ('9', 'Spades'), ('9', 'Spades'), ('Jack', 'Hearts'), ('9', 'Clubs')], Rank: 8


# Answer of 1b here

In the above question we have used the Simulated Annealing algorithm. What are the pros and cons of using this approach? Please give at least two of each (5 marks)

pros of simulated anealing:
1. It is stochastic and it can easily escape the local minima
2. it is very versatile and it can be applied to wide range of problems 

cons of simulated anealing:   
1. It is very slow just like me
2. It does not gurantee optimal solution because it is heuristic algo
3. Personally i don't like it     



# Answer of 1c here

What other algorithm(s) can we use here and what advantages and disadvantages would we get from them. You need to name at least one algorithm and give at least two advantages and disadvantages. (5 marks)

Genetic Algorithm use kar saktay

Advantages:   
1. It can find global optimal solution 
2. It can handle large spaces 

Disadvantages:   
1. It is very complex 
2. It can get stuck at local optima 


